In [ ]:
# Install local biota to overwrite system installation

In [ ]:
%%capture
!pip install biota/

In [ ]:
import os
import getpass
import numpy as np
from pathlib import Path
from functools import partial
import gdal

import biota
from biota import download as dw

from ipyleaflet import Marker
from ipywidgets import Output, Layout, HTML
from traitlets import (
    Int, Bool, Float, link, Unicode, observe, List,
    CInt, CFloat
)
import ipyvuetify as v
from sepal_ui import mapping as m
from sepal_ui import sepalwidgets as sw

from component.widget.custom_widgets import *

In [ ]:
HEADER_TITLE = "SEPAL BIOTA (BETA)"
HEADER_TEXT = """
The BIOmass Tool for Alos (BIOTA) was developed by LTS International and the University of Edinburgh to calculate above-ground biomass from L-band satallite data in dry forests and savanhas, as well as biomass change and forest degradation.
"""
OUTPUT_TITLE = "OUTPUT PROCESS"

In [ ]:
import functools
def loading(btn, alert):
    def decorator_loading(func):
        @functools.wraps(func)
        def wrapper_loading(*args, **kwargs):
            btn.loading=True
            try:
                value = func(*args, **kwargs)
            except Exception as e:
                btn.loading=False
                alert.add_msg(f'{e}', type_='error')
                raise e
            btn.loading=False
            return value
        return wrapper_loading
    return decorator_loading

In [ ]:
def round_(x, grid):
    return grid * round(x/grid)

def assert_errors(self, error):
    self.w_alert.add_msg(error, type_='error')
    return error

In [ ]:
class Tabs(v.Card):
    
    current = Int(0).tag(sync=True)
    
    def __init__(self, titles, content, **kwargs):
        
        self.background_color="primary"
        self.dark = True
        
        self.tabs = [v.Tabs(v_model=self.current, children=[
            v.Tab(children=[title], key=key) for key, title in enumerate(titles)
        ])]
        
        self.content = [v.TabsItems(
            v_model=self.current, 
            children=[
                v.TabItem(children=[content], key=key) for key, content in enumerate(content)
            ]
        )]
        
        self.children= self.tabs + self.content
        
        link((self.tabs[0], 'v_model'),(self.content[0], 'v_model'))
        
        super().__init__(**kwargs)
        
        

In [ ]:
class Optional(v.Card):
    
    lee_filter = Bool(True).tag(sync=True)
    speckle_filter = Bool(True).tag(sync=True)
    
    downsample_factor = CInt(1).tag(sync=True)
    window_size = CInt(5).tag(sync=True)
    forest_threshold = CFloat(10.).tag(sync=True)
    area_threshold = CFloat(0.).tag(sync=True)
    
    change_area_threshold = CInt(2).tag(sync=True)
    change_magnitude_threshold = CInt(15).tag(sync=True)
    
    contiguity = Unicode('queen').tag(sync=True)
    sm_interpolation = Unicode('average').tag(sync=True)
    polarisation = Unicode('HV').tag(sync=True)
    
    
    def __init__(self, **kwargs):
    
        super().__init__(**kwargs)
        
        w_lee_filter = v.Checkbox(label='Lee filter', v_model=self.lee_filter)
        w_speckle_filter = v.Checkbox(label='Speckle filter', v_model=self.lee_filter)
        
        w_downsample_factor = v.TextField(label='Downsample factor', type='number', v_model=self.downsample_factor)
        w_window_size = v.TextField(label='Window size', type='number', v_model=self.window_size)
        w_forest_threshold = v.TextField(label='Forest threshold', type='number', v_model=self.forest_threshold)
        w_area_threshold = v.TextField(label='Area threshold', type='number', v_model=self.area_threshold)
        
        w_change_area_threshold = v.TextField(label='Change area threshold', type='number', v_model=self.change_area_threshold)
        w_change_magnitude_threshold = v.TextField(label='Change magnitude threshold', type='number', v_model=self.change_magnitude_threshold)
        
        w_contiguity = v.Select(label='Contiguity', items=['rook', 'queen'], v_model=self.contiguity)
        w_sm_interpolation = v.Select(label='SM interpolation', items=['nearest', 'average', 'cubic'], v_model=self.sm_interpolation)
        w_polarisation = v.Select(label='Polarisation', items=['HV', 'HH', 'VV', 'VH'], v_model=self.polarisation)
        
        link((w_lee_filter, 'v_model'),(self, 'lee_filter'))
        link((w_speckle_filter, 'v_model'),(self, 'speckle_filter'))
        
        link((w_downsample_factor, 'v_model'),(self, 'downsample_factor'))
        link((w_window_size, 'v_model'),(self, 'window_size'))
        link((w_forest_threshold, 'v_model'),(self, 'forest_threshold'))
        link((w_area_threshold, 'v_model'),(self, 'area_threshold'))
        
        link((w_change_area_threshold, 'v_model'),(self, 'change_area_threshold'))
        link((w_change_magnitude_threshold, 'v_model'),(self, 'change_magnitude_threshold'))
        
        link((w_contiguity, 'v_model'),(self, 'contiguity'))
        link((w_sm_interpolation, 'v_model'),(self, 'sm_interpolation'))
        link((w_polarisation, 'v_model'),(self, 'polarisation'))
        
        self.children=[
            w_lee_filter,
            w_speckle_filter,
            w_downsample_factor,
            w_window_size,
            w_forest_threshold,
            w_area_threshold,
            w_change_area_threshold,
            w_change_magnitude_threshold,
            w_contiguity,
            w_polarisation,
            w_sm_interpolation,
        ]

In [ ]:
class Required(v.Card):

    # Initial widgets
    lat = Float(0).tag(sync=True)
    lon = Float(-75).tag(sync=True)
    year_1 = Unicode().tag(sync=True)
    year_2 = Unicode().tag(sync=True)
    large_tile = Bool(True).tag(sync=True)
    grid = Int(5).tag(sync=True)
    
    def __init__(self, **kwargs):
    
        super().__init__(**kwargs)
    
        # 1. .Input widgets (Parameters)
        w_lat = v.TextField(disabled=True, label='Latitude',v_model=self.lat,)
        w_lon = v.TextField(disabled=True, label='Longitude', v_model=self.lon,)
        w_year_1 = v.TextField(label='Year 1', type='number', v_model=self.year_1)
        w_year_2 = v.TextField(label='Year 2', type='number',v_model=self.year_2)
        w_lg_tile = v.Checkbox(label='Large Tile', v_model=self.large_tile)
        w_grid = v.RadioGroup(v_model=self.grid,children=[
            v.Radio(label="1x1 grid", value=1),
            v.Radio(label="5x5 grid", value=5)
        ])
        
        self.w_download = sw.Btn('Download images', class_='pl-5')
        
        link((w_lon, 'v_model'), (self, 'lon'))
        link((w_lat, 'v_model'), (self, 'lat'))
        link((w_year_1, 'v_model'), (self, 'year_1'))
        link((w_year_2, 'v_model'), (self, 'year_2'))
        link((w_lg_tile, 'v_model'), (self, 'large_tile'))
        link((w_grid, 'v_model'), (self, 'grid'))
        
        self.children=[
                w_lon, 
                w_lat,
                w_year_1,
                w_year_2,
                w_lg_tile,
                w_grid,
                self.w_download
        ]


In [ ]:
class Parameters(v.Layout):
    
    def __init__(self, map_=None, **kwargs):
        
        # Widget classes
        self.class_ = "flex-column pa-2"
        self.row = True
        self.xs12 = True


        super().__init__(**kwargs)
        
        # Parameters
        
        self.map_ = map_
        
        # Set-up workspace
        self.USER = getpass.getuser()
        self._workspace()
        self.PARAMETER_FILE = os.path.join(os.getcwd(), 'biota/cfg/McNicol2018.csv')
        
        # Set up process parameters
        self.required = Required(class_='pa-4')
        self.optional = Optional(class_='pa-4')
        
        # Alerts
        self.w_alert = Alert(children=['Select parameters']).show()
        self.ou_progress = Output()

        # Events
        self.required.w_download.on_event('click', self._download_event)

        self.children = [
            
            v.Card(children=[
                v.CardTitle(children=[HEADER_TITLE]), 
                v.CardText(children=[HEADER_TEXT])
            ]),
            
            v.Row(class_="d-flex flex-row ", xs12=True, md6=True,
               children=[
                    v.Col(children=[
                        Tabs(['Required inputs', 'optional inputs'],
                             [self.required, self.optional])
                    ]),
                    v.Col(
                        children=[
                            v.Card(class_='pa-2 justify-center', children=[
                                map_
                            ])
                        ]
                    ),
            ]),
            v.Card(class_="flex-row pa-2 mb-3", children=[
                self.w_alert,
            ]),
        ]
        
        # Decorate self._download() method
        self._download = loading(self.required.w_download, self.w_alert)(self._download)
        
    def _workspace(self):
        """ Creates the workspace necessary to store the data

        return:
            Returns environment Paths

        """

        base_dir = Path(os.path.join('/home', self.USER))

        root_dir = base_dir/'module_results/smfm'
        data_dir = root_dir/'data'
        output_dir = root_dir/'outputs'

        root_dir.mkdir(parents=True, exist_ok=True)
        data_dir.mkdir(parents=True, exist_ok=True)
        output_dir.mkdir(parents=True, exist_ok=True)

        self.root_dir = root_dir
        self.data_dir  = data_dir
        self.output_dir = output_dir

    def _download_event(self, *args):
        
        years = [int(year) for year in [self.required.year_1, self.required.year_2] if year]
        lat = round_(self.required.lat, self.required.grid)
        lon = round_(self.required.lon, self.required.grid)
        assert (years != []), assert_errors(self, 'Please select at least one year.')
        
        for y in years:
            self._download(lat, lon, y)
            
    def _download(self, *args):
        
        lat, lon, y = args
        self.w_alert.add_msg(f'Downloading year {y} for lat: {lat}, lon: {lon}...', type_='info')
        dw.download(lat,lon,y,
                    large_tile=self.required.large_tile, 
                    output_dir=self.data_dir, 
                    verbose=True)
        self.w_alert.add_msg(f'Decompressing year {y}...', type_='info')
        self._decompress()
        
        self.w_alert.add_msg(f'Done {years} for lat: {lat}, lon: {lon}!', type_='info')

    def _decompress(self):
        
        tar_files = list(self.data_dir.glob('*.tar.gz'))

        for tar in tar_files:
            if not os.path.exists(os.path.join(self.data_dir, tar.name[:-7])):
                self.w_alert.add_msg(f'Decompressing {tar.name}...', type_='info')
                dw.decompress(str(tar))
                self.w_alert.add_msg(f'All the images were succesfully unzipped.', type_='success')

In [ ]:
class Process(v.Card):

    # Process widgets
    forest_p = Bool(False).tag(sync=True)
    forest_ch = Bool(False).tag(sync=True)
    gamma0 = Bool(True).tag(sync=True)
    biomass = Bool(True).tag(sync=True)
    biomass_ch = Bool(False).tag(sync=True)
    forest_cv = Bool(False).tag(sync=True)
    
    # Outputs
    true_cb = List([]).tag(sync=True)
    
    def __init__(self, parameters, **kwargs):
        
        self.param = parameters
        self.tile_1 = None
        self.tile_2 = None
        self.change_tile = None
        
        self.gamma0_tile = None
        self.agb_tile = None
        self.biomass_change_tile = None
        self.forest_change_code = None
        
        self._observe_forest_p()
        
        super().__init__(**kwargs)
        
        
        self.w_alert = Alert(children=['Select process']).show()
        
        w_forest_p = v.Checkbox(label='Forest property', class_='pl-5', v_model=self.forest_p, disabled=True)
        w_forest_ch = v.Checkbox(label='Change type', class_='pl-5', v_model=self.forest_ch)
        w_gamma0 = v.Checkbox(label='Gamma0', class_='pl-5', v_model=self.gamma0)
        w_biomass = v.Checkbox(label='Biomass', class_='pl-5', v_model=self.biomass)
        w_biomass_ch = v.Checkbox(label='Biomass change', class_='pl-5', v_model=self.biomass_ch)
        w_forest_cv = v.Checkbox(label='Forest cover', class_='pl-5', v_model=self.forest_cv, disabled=True)
        
        # Output widgets
        
        self.ou_display = Output()

        self.w_select_output = v.Select(items=self.true_cb, v_model=None, label='Select process')
        self.btn_process = sw.Btn('Get outputs', class_='pl-5')
        
        self.btn_add_map = sw.Btn('Display', class_='pl-5')
        self.btn_write_raster = sw.Btn('Write raster', class_='ml-5')
        
        # Linked widgets

        link((w_forest_p, 'v_model'), (self, 'forest_p'))
        link((w_forest_ch, 'v_model'), (self, 'forest_ch'))
        link((w_gamma0, 'v_model'), (self, 'gamma0'))
        link((w_biomass, 'v_model'), (self, 'biomass'))
        link((w_biomass_ch, 'v_model'), (self, 'biomass_ch'))
        link((w_forest_cv, 'v_model'), (self, 'forest_cv'))
        link((self.w_select_output, 'items'), (self, 'true_cb'))
        
        self.btn_process.on_event('click', partial(self._event, func=self._process))
        self.btn_add_map.on_event('click', partial(self._event, func=self._display))
        self.btn_write_raster.on_event('click', partial(self._event, func=self._write_raster))
        
        self.children=[v.Card(children=[
                v.CardTitle(children=[OUTPUT_TITLE]),
                v.CardText(class_="d-flex flex-row", children=[
                    v.Col(children=[w_forest_p,w_forest_ch,]),
                    v.Col(children=[w_gamma0,w_biomass,]),
                    v.Col(children=[w_forest_cv, w_biomass_ch,]),
                    self.btn_process
                ])
            ]),
            self.w_alert,
            v.Row(class_="d-flex flex-row ", xs12=True, md6=True,
               children=[
                    v.Col(
                        children=[
                            v.Card(class_='pa-4',
                                   children=[
                                       self.w_select_output, 
                                       self.btn_add_map,
                                       self.btn_write_raster])]),
                    v.Col(
                        children=[
                            v.Card(class_='pa-2 justify-center', 
                                   children=[self.ou_display])]),
            ]),
        ]

        # Add all True checkBoxes to a List
        # Inspect its change
    
        # Decorate loading functions
        self._write_raster = loading(self.btn_write_raster, self.w_alert)(self._write_raster)
        self._process = loading(self.btn_process, self.w_alert)(self._process)
        self._display = loading(self.btn_add_map, self.w_alert)(self._display)
    
    
    def _event(self, widget, event, data, func):
        """ This function is used to execute decorated 
        functions with an event
        
        example:
            
            btn.on_event('click', partial(self._event, func=self._process))
            where self._process is the decorated function to be executed.
        
        """
        func()
    
    @observe('forest_p', 'forest_ch', 'gamma0', 
             'biomass', 'biomass_ch', 'forest_cv')
    def _observe_forest_p(self, change=None):
        labels = {
            'Forest property': self.forest_p,
            'Change type': self.forest_ch,
            'Gamma0': self.gamma0,
            'Biomass': self.biomass,
            'Biomass change': self.biomass_ch,
            'Forest cover': self.forest_cv
        }

        self.true_cb = [k for k, v in labels.items() if v is True]
        
    
    def _validate_inputs(self):
        
        assert (self.param.required.year_1 != '') or (self.param.required.year_2 != ''), assert_errors(self, 'Please select at least one year')
        # Test that inputs are of reasonable lats/lons/years
        assert self.param.required.lat < 90. or self.param.required.lat > -90., assert_errors(self, "Latitude must be between -90 and 90 degrees.")
        assert self.param.required.lon < 180. or self.param.required.lon > -180., assert_errors(self, "Longitude must be between -180 and 180 degrees.")
        
        assert self.param.optional.downsample_factor >= 1 and type(self.param.optional.downsample_factor) == int, assert_errors(self, "Downsampling factor must be an integer greater than 1.")
        assert type(self.param.optional.lee_filter) == bool, assert_errors(self, "Option lee_filter must be set to 'True' or 'False'.")
        assert type(self.param.optional.window_size) == int, assert_errors(self, "Option window_size must be an integer.")
        assert self.param.optional.window_size % 2 == 1, assert_errors(self, "Option window_size must be an odd integer.")
        assert self.param.optional.contiguity in ['rook', 'queen'], assert_errors(self, "Contiguity constraint must be 'rook' or 'queen'.")
        assert self.param.optional.sm_interpolation in ['nearest', 'average', 'cubic'], assert_errors(self, "Soil moisture interpolation type must be one of 'nearest', 'average' or 'cubic'.")

        assert type(self.param.optional.forest_threshold) == float or type(self.param.optional.forest_threshold) == int, assert_errors(self, "Forest threshold must be numeric.")
        assert type(self.param.optional.area_threshold) == float or type(self.param.optional.area_threshold) == int, assert_errors(self, "Area threshold must be numeric.")
        
        # Verify at least one process is selected
        
        
    def _load_tile(self, year):

        try:
            tile = biota.LoadTile(str(self.param.data_dir), 
                                       round_(self.param.required.lat, self.param.required.grid), 
                                       round_(self.param.required.lon, self.param.required.grid), 
                                       year,
                                       parameter_file = self.param.PARAMETER_FILE,
                                       lee_filter = self.param.optional.lee_filter, 
                                       forest_threshold = self.param.optional.forest_threshold, 
                                       area_threshold = self.param.optional.area_threshold, 
                                       output_dir = str(self.param.output_dir))
            return tile
        
        except Exception as e:
            
            self.w_alert.add_msg(f'{e}', type_='error')
            raise
    
    def _process(self):
        """Event trigger when btn_process is clicked
        
        * This function is decorated by loading
        
        """
        
        # Raise error if validation doesn't pass
        self._validate_inputs()
        
        # Create tiles if years are selected.
        if self.param.required.year_1 : self.tile_1 = self._load_tile(int(self.param.required.year_1))
        if self.param.required.year_2 : self.tile_2 = self._load_tile(int(self.param.required.year_2))
                
        for process in self.true_cb:
            self.w_alert.reset()
            if process == 'Gamma0':
                self.w_alert.type_='info'
                self.w_alert.append_msg(f'Computing Gamma0')
                self.gamma0_tile = self.tile_1.getGamma0(polarisation = self.param.optional.polarisation, units = 'decibels')
                self.w_alert.append_msg(f'Gamma0 computed and ready to display')
                self.w_alert.type_='success'

            elif process == 'Biomass':
                self.w_alert.append_msg(f'Computing Biomass')
                self.w_alert.type_='info'
                self.agb_tile = self.tile_1.getAGB()
                self.w_alert.append_msg(f'Biomass computed and ready to display')

            elif process in ['Biomass change', 'Change type']:

                self.w_alert.add_msg(f'Retrieving change tile...')
                assert all((self.param.required.year_1 , self.param.required.year_2)), assert_errors(self, "To calculate change, both years has to be filled")
                # Compute change tile
                self.change_tile = biota.LoadChange(
                    self.tile_1, 
                    self.tile_2,
                    change_area_threshold = self.param.optional.change_area_threshold, 
                    change_magnitude_threshold = self.param.optional.change_magnitude_threshold,
                    contiguity = self.param.optional.contiguity
                )

                if process == 'Biomass change':
                    self.w_alert.add_msg(f'Computing Biomass Change')
                    self.w_alert.type_='info'
                    self.biomass_change_tile = self.change_tile.getAGBChange()
                    self.w_alert.append_msg(f'Biomass change tile computed and ready to display')

                elif process == 'Change type':
                    self.w_alert.add_msg(f'Computing Change Type')
                    self.w_alert.type_='info'
                    self.change_tile.getChangeType()
                    self.forest_change_code = self.change_tile.ChangeCode
                    self.w_alert.append_msg(f'Change type tile computed and ready to display')

                self.w_alert.type='success'
            else:
                self.w_alert.append_msg(f'Check at least one process to compute')
                        
    def _validate_display_dwn(self):
        
        if not self.w_select_output.v_model:
            # No selection
            self.w_alert.add_msg(f'Select at least one process to download', type_='warning')
            # Raise a fake error to stop the execution, voilá won't display this
            raise
            
    def _write_raster(self):
        """Write processed raster
        
        * This function is decorated by loading
        
        Args:
            widget (ipywidgets): w_select_output with list of possible processed rasters (self.TILES)
                                
        """
        
        self._validate_display_dwn()
        
        TILES = {
            # Add new tiles when are avaiable
            'Gamma0': self.gamma0_tile,
            'Biomass': self.agb_tile,
            'Biomass change': self.biomass_change_tile,
            'Change type': self.forest_change_code
        }
        
        # Get current raster tile name
        tile_name = self.w_select_output.v_model
        
        # Get tile from selected dropdown
        tile = TILES[tile_name]
        assert (tile is not None), assert_errors(self, f'Before to write, you have to calculate {tile_name}')
        
        if tile_name in ['Biomass', 'Gamma0', 'Biomass change']:
            self.tile_1._LoadTile__outputGeoTiff(tile, tile_name)

        elif tile_name in ['Change type']:
            self.tile_1._LoadTile__outputGeoTiff(tile, tile_name, dtype = gdal.GDT_Byte)

        self.w_alert.add_msg(f'{tile_name} succesfully exported in {self.param.output_dir}', type_='success')

        
    def _display(self):
        """Display processed raster.
        
        * This function is decorated by loading
        
        Args:
            widget (ipywidgets): w_select_output with list of possible processed rasters (self.TILES)
                                
        """
        TILES = {
            # Add new tiles when are avaiable
            'Gamma0': self.gamma0_tile,
            'Biomass': self.agb_tile,
            'Biomass change': self.biomass_change_tile,
            'Change type': self.forest_change_code
        }
        
        self._validate_display_dwn()
        
        # Get current raster tile name
        tile_name = self.w_select_output.v_model
        
        # Get tile from selected dropdown
        tile = TILES[tile_name]
        assert (tile is not None), assert_errors(self, f'Before to display, you have to calculate {tile_name}')
        
        with self.ou_display:
            self.ou_display.clear_output()
            if tile_name == 'Biomass':
                title, cbartitle, vmin, vmax, cmap = 'AGB', 'tC/ha', 0, 40, 'YlGn'
            elif tile_name == 'Gamma0':
                title, cbartitle, vmin, vmax, cmap = f'Gamma0 {self.param.optional.polarisation}', \
                                                    'decibels', -20, -10, 'Greys_r'
            elif tile_name == 'Biomass change':
                title, cbartitle, vmin, vmax, cmap = 'AGB Change', 'tC/ha', -10, 10, 'YlGn'
            
            elif tile_name == 'Change type':
                # Hide minor gain, minor loss and nonforest in display output
                change_code_display = np.ma.array(
                    tile, 
                    mask = np.zeros_like(tile, dtype = np.bool)
                )
                change_code_display.mask[np.isin(change_code_display, [0, 3, 4, 255])] = True
                
                # Overwrite current tile with new change_code_display
                tile, title, cbartitle, vmin, vmax, cmap = change_code_display, 'Change type', 'Type', 1, 6, 'Spectral'

            # Show arrays with showArray method from LoadTile object
            # We are just using this method to display any tile with the given UI.lat and UI.lon
            self.tile_1._LoadTile__showArray(tile, title, cbartitle, vmin, vmax, cmap)
        

In [ ]:
def _return_coordinates(self, **kwargs):
    if kwargs.get('type') == 'click':

        # Remove markdown if there is one
        map_.remove_last_layer()

        lat, lon = kwargs.get('coordinates')

        map_.add_layer(Marker(location=kwargs.get('coordinates')))
        parameters.required.lat = round(lat,2)
        parameters.required.lon = round(lon,2)
# Map
map_ = m.SepalMap()
parameters = Parameters(map_=map_)
map_.on_interaction(partial(_return_coordinates, parameters))

In [ ]:
process = Process(parameters)

In [ ]:
# parameters

In [ ]:
# process

In [ ]:
process_tile = sw.Tile(id_='process', title='Process', inputs=[process])

parameters_tile = sw.Tile(id_='parameters', title='Parameters', inputs=[parameters])

appBar = sw.AppBar('SMFM Biota')

content = [
    parameters_tile,
    process_tile,
]

#create a drawer 
item_param = sw.DrawerItem('Parameters', 'mdi-map-marker-check', card="parameters").display_tile(content)
item_process = sw.DrawerItem('Process', 'mdi-earth', card="process").display_tile(content)

code_link = 'https://github.com/ingdanielguerrero/smfm_biota'
wiki_link = 'https://github.com/ingdanielguerrero/smfm_biota/blob/main/README.md'
issue = 'https://github.com/ingdanielguerrero/smfm_biota/issues/new'

items = [
    item_param,
    item_process,
]

drawer = sw.NavDrawer(items, code = code_link, wiki = wiki_link, issue = issue).display_drawer(appBar.toggle_button)

#build the app 
app = sw.App(
    tiles=content, 
    navDrawer=drawer
).show_tile('parameters')
#display the app
app